# Sampling Algorithm
- `Sampler`는 기본적으로 제안된 매개변수 값과 평가된 목표 값의 기록을 사용, 검색 공간을 지속적으로 좁혀줌
- `Sampler`가 매개변수를 제안하는 방법은 [BaseSampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.BaseSampler.html#optuna.samplers.BaseSampler)에 들어가 있음
    - `Relative Sampling` : 샘플링 알고리즘이 매개변수 간의 상관관계를 사용할 수 있도록 **여러 매개변수의 값을 동시에 결정**함
        - 대상 매개변수는 `infer_relative_search_space()`**에 의해 결정되는 상대 검색 공간으로 기술**됨
    - `Independent Sampling` : **매개변수 간의 관계를 고려하지 않고 단일 매개변수의 값을 결정**함. 
    
    - Trial 시작 시 `infer_relative_search_space()`가 호출됨 : 그 다음 `sample_relative()`가 호출되어 상대 검색 공간에서 매개변수를 샘플링함
    - 목적 함수를 실행하는 동안 `sample_independent()`는 상대 검색 공간에 속하지 않는 매개변수를 샘플링하는 데 사용됨

- 샘플링 알고리즘엔 요런 것들이 있다
    - `GridSampler`
    - `RandomSampler`
    - `TPESampler` : Tree-structured Parzen Estimator
    - `CmaEsSampler` : CMA-ES
    - `PartialFixedSampler` : Algorithm to enable partial fixed parameters
    - `NSGAIISampler` : Nondominated Sorting Genetic Algorithm II
    - `QMCSampler` : A Quasi Monte Carlo sampling algorithm

- 디폴트는 `TPESampler`를 사용함

In [1]:
import optuna

In [2]:
study = optuna.create_study()
print(f"Sampler is {study.sampler.__class__.__name__}")

[I 2022-12-10 15:41:14,254] A new study created in memory with name: no-name-1a539655-99c0-43f9-b9b0-72d61cd373c9


Sampler is TPESampler


In [3]:
# sampler는 study를 정의할 떄 지정함
study = optuna.create_study(sampler = optuna.samplers.RandomSampler())
print(f"Sampler is {study.sampler.__class__.__name__}")

study = optuna.create_study(sampler = optuna.samplers.CmaEsSampler())
print(f"Sampler is {study.sampler.__class__.__name__}")


[I 2022-12-10 15:42:20,424] A new study created in memory with name: no-name-6b7e8ad4-bd47-473f-83ad-6fd2a100ef26
[I 2022-12-10 15:42:20,429] A new study created in memory with name: no-name-6c436f61-2653-4828-8c76-1ff6c3992e81


Sampler is RandomSampler
Sampler is CmaEsSampler


# Pruning Algorithm

- `Pruner`는 훈련 초기 단계에서 유망하지 못한 `Trial`들을 자동적으로 멈춤
- 요런 것들이 있다
    - `MedianPruner`
    - `NopPruner`
    - `PatientPruner`
    - `PercentilePruner`
    - `SuccessiveHalvingPruner`
    - `HyperBandPruner`
    - `ThresholdPruner`

- 예시에서는 `MedianPruner`를 사용하지만 `SuccessiveHalvingPruner`, `HyperbandPruner`가 벤치마크 결과에선 가장 좋은 성능을 냈다.

### Pruner 이용하기

In [5]:
import logging
import sys

import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection

def objective(trial):
    iris = sklearn.datasets.load_iris()
    classes = list(set(iris.target))
    
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        iris.data, iris.target, test_size = 0.25, random_state = 0
    )
    
    alpha = trial.suggest_float('alpha', 1e-5, 1e-1, log = True)
    clf = sklearn.linear_model.SGDClassifier(alpha = alpha)
    
    for step in range(100):
        clf.partial_fit(train_x, train_y, classes = classes) # 1번의 에포크만을 수행함
        
        # 중간값 report
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, step)
        
        # 중간값에 의해 Pruning을 함 : 그 기준은 알고리즘으로 따로 있는 듯
        if trial.should_prune():
            raise optuna.TrialPruned()
            
    return 1.0 - clf.score(valid_x, valid_y)

In [8]:
# Pruner는 함수 바깥에서 정의함
# optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))

study = optuna.create_study(pruner = optuna.pruners.MedianPruner())
study.optimize(objective, n_trials = 20)

[I 2022-12-10 15:56:38,176] A new study created in memory with name: no-name-8e7b50fa-5d7c-46a2-93e2-b8f04103d41b


A new study created in memory with name: no-name-8e7b50fa-5d7c-46a2-93e2-b8f04103d41b


[I 2022-12-10 15:56:38,668] Trial 0 finished with value: 0.07894736842105265 and parameters: {'alpha': 0.0003734747131584509}. Best is trial 0 with value: 0.07894736842105265.


Trial 0 finished with value: 0.07894736842105265 and parameters: {'alpha': 0.0003734747131584509}. Best is trial 0 with value: 0.07894736842105265.


[I 2022-12-10 15:56:39,030] Trial 1 finished with value: 0.39473684210526316 and parameters: {'alpha': 3.926612831807714e-05}. Best is trial 0 with value: 0.07894736842105265.


...

Trial 17 pruned. 


[I 2022-12-10 15:56:41,098] Trial 18 pruned. 


Trial 18 pruned. 


[I 2022-12-10 15:56:41,702] Trial 19 finished with value: 0.052631578947368474 and parameters: {'alpha': 0.0015926384389243736}. Best is trial 3 with value: 0.02631578947368418.


Trial 19 finished with value: 0.052631578947368474 and parameters: {'alpha': 0.0015926384389243736}. Best is trial 3 with value: 0.02631578947368418.


## Sampler와 Pruner의 조합
- 딥러닝에 쓰이지 않을 때 벤치마크 결과가 있다.
    - `RandomSampler` + `MedianPruner`
    - `TPESampler` + `HyperBandPruner`
    
- 딥러닝에선 어떨까?
    - 병렬처리 제한됐을 때 : `TPE` / 저차원&연속적 차원이라면 `GP-EI`
    - 병렬처리 충분할 때
        - 범주/조건 하이퍼파라미터 없을 경우 : `CMA-ES`, `Random Search`
        - 범주/조건 하이퍼파라미터 있을 경우 : `Random Search` or `Genetic Algorithm`
        
        
- **범주/조건 하이퍼파라미터가 무슨 얘기**임?

### Pruning을 위한 통합 알고리즘

In [10]:
pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
bst = xgb.train(param, dtrain, evals = [(dvalid, 'validation')], callbacks = [pruning_callback])

NameError: name 'trial' is not defined